## Getting started with Snakemake workflows for automated bioinformatics analysis.

## Author: Ali Pirani

## Software we're going to use

We're going to be using [conda](https://conda.io/en/latest/) and [snakemake](https://snakemake.readthedocs.io/en/stable/), as well as packages from [bioconda](https://bioconda.github.io). 

If you wanted to run all of this on your own computer, you'll need to follow the bioconda install instructions.

We'll be implementing a short read quality check and trimming pipeline, using [fastqc](https://www.bioinformatics.babraham.ac.uk/projects/fastqc/), [trimmomatic](http://www.usadellab.org/cms/?page=trimmomatic), and [multiqc](https://multiqc.info/) to demonstrate how to get started with the Snakemake workflow. 

You can see the full set of installed software requirements in a conda `environment.yml` file located under binder directory. More on binder later.

You could use this install file to run everything we're doing today on your laptop, with: 
```
conda env create --file binder/environment.yml -n smake
conda activate smake
pip install bash_kernel
python -m bash_kernel.install
```

Why use a workflow management tool?

- Dependency management
- Reentrancy - start back up where you left off
- Reusable
- Documented
- Portable

There are many to choose from: https://github.com/pditommaso/awesome-pipeline

## Tasks we are going to do

- Download data which we will use for Snakemake workflow
- Create a Snakefile to write our first rule i.e running FastQC on fastq reads.
- 

## Download some data

Execute the below cell to fetch some data. This will download the example fastq data to your data directory.

Note: To run bash commands, select bash kernel from the top right corner.

In [1]:
curl -L https://github.com/ctb/2019-snakemake-ucdavis/raw/9db09bc0b6a3469f8a0d4996d4b2995bf36e5d27/data/0Hour_001_1.fq.gz > data/0Hour_001_1.fq.gz
curl -L https://github.com/ctb/2019-snakemake-ucdavis/raw/9db09bc0b6a3469f8a0d4996d4b2995bf36e5d27/data/6Hour_001_1.fq.gz > data/6Hour_001_1.fq.gz
curl -L https://github.com/ctb/2019-snakemake-ucdavis/raw/9db09bc0b6a3469f8a0d4996d4b2995bf36e5d27/data/0Hour_001_2.fq.gz > data/0Hour_001_2.fq.gz
curl -L https://github.com/ctb/2019-snakemake-ucdavis/raw/9db09bc0b6a3469f8a0d4996d4b2995bf36e5d27/data/6Hour_001_2.fq.gz > data/6Hour_001_2.fq.gz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   190  100   190    0     0    688      0 --:--:-- --:--:-- --:--:--   688
100 7689k  100 7689k    0     0  11.4M      0 --:--:-- --:--:-- --:--:-- 11.4M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   190  100   190    0     0    917      0 --:--:-- --:--:-- --:--:--   917
100 5262k  100 5262k    0     0   9.8M      0 --:--:-- --:--:-- --:--:-- 16.5M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   190  100   190    0     0    745      0 --:--:-- --:--:-- --:--:--   745
100 8160k  100 8160k    0     0  14.7M      0 --:--:-- --:--:-- --:--:-- 14.7M
  % Total    % Received % Xferd  Average Speed   Tim

In [2]:
ls data/

0Hour_001_1.fq.gz  0Hour_001_2.fq.gz  6Hour_001_1.fq.gz  6Hour_001_2.fq.gz


Lets activate the snakemake environment using conda.

In [5]:
conda activate smake

(smake) 

: 1

## Running snakemake!

### Create your first Snakefile.

We will define our rules of workflow in a Snakefile. 

Create a new text file by selecting (`File`, `New`, `Text file`) from the the top left corner menu and copy/paste this first rule:

```
rule fastqc_a_file:
  shell:
    "fastqc data/0Hour_001_1.fq.gz"
```

Save and Rename it to "Snakefile". (Right click on the file from the file explorer in left panel and select Rename)

* the snakemake configuration file is by default called `Snakefile`

Now, run snakemake:


In [ ]:
snakemake

What was the error?

Lets run it with 1 core.

In [ ]:
snakemake --cores 1

You should see:

```

Building DAG of jobs...
Using shell: /bin/bash
Provided cores: 1 (use --cores to define parallelism)
Rules claiming more threads will be scaled down.
Job stats:
job              count    min threads    max threads
-------------  -------  -------------  -------------
fastqc_a_file        1              1              1
total                1              1              1

Select jobs to execute...

[Wed Nov  3 15:49:22 2021]
rule fastqc_a_file:
    jobid: 0
    resources: tmpdir=/tmp

Started analysis of 0Hour_001_1.fq.gz
Approx 5% complete for 0Hour_001_1.fq.gz
...
Analysis complete for 0Hour_001_1.fq.gz
[Wed Nov  3 15:49:27 2021]
Finished job 0.
1 of 1 steps (100%) done
Complete log: /home/apirani/Session8_snakemake/.snakemake/log/2021-11-03T154921.245311.snakemake.log

(smake) 
```

### Updating the Snakefile to track inputs and outputs 

At the moment this is basically just a shell script with extra syntax... what's the point?

Well, shell scripts - and this snakefile, too - will rerun the command every time you run the file, even if there's no reason to do so because the file hasn't changed.

**Digression:** This is particularly important for large or long workflows, where you're dealing with 10s to 100s of files that may take hours to days to process! It can be hard to figure out which files to rerun, but (spoiler alert) snakemake can really help you do this!

It's hard to track this kind of thing in a shell script - I usually just comment out the lines I don't want run, or break my commands up into multiple shell scripts so they don't take so long - but with snakemake, you can annotate the rule with input and output files! 

If you annotate the rule with input/output files, Snakemake will automatically check if the output file exists and makes a decision to either rerun or do nothing.

We will see how it does that in the following section.

Change your snakefile to look like this:

```
rule fastqc_a_file:
  input:
    "data/0Hour_001_1.fq.gz"
  output:
    "data/0Hour_001_1_fastqc.html",
    "data/0Hour_001_1_fastqc.zip"
  shell:
    "fastqc data/0Hour_001_1.fq.gz"
```

here, we've annotated the rule with the required
**input** file, as well as the expected **output** files.

Question: how do we know what the output files are?

Now run:


In [ ]:
snakemake --cores 1

You should see something like this:

```
Building DAG of jobs...
Nothing to be done.
Complete log: /home/apirani/Session8_snakemake/.snakemake/log/2021-11-03T155832.508508.snakemake.log
```

What happened??

snakemake looked at the file, saw that the output files existed, and figured out that it didn't need to do anything!

### Forcibly re-running things

You can tell snakemake to run the rule no matter what with `-f`:

In [ ]:
snakemake -f --cores 1

You can also remove an output file and it will automatically re-run:

In [ ]:
rm data/*.html
snakemake --cores 1

Note that you don't need to remove *all* the output files to rerun a command - just remove *one* of them.

You can *also* update the timestamp on an *input* file, and snakemake will figure out that the output file is older than the input file, and rerun things.

In [ ]:
touch data/*.fq.gz
snakemake --cores 1

This feature of Snakemake will become importanta later.

### Multiple rules

Let's add a rule to run fastqc on a second file:

```
rule fastqc_a_file:
  input:
    "data/0Hour_001_1.fq.gz"
  output:
    "data/0Hour_001_1_fastqc.html",
    "data/0Hour_001_1_fastqc.zip"
  shell:
    "fastqc data/0Hour_001_1.fq.gz"

rule fastqc_a_file2:
  input:
    "data/6Hour_001_1.fq.gz"
  output:
    "data/6Hour_001_1_fastqc.html",
    "data/6Hour_001_1_fastqc.zip"
  shell:
    "fastqc data/6Hour_001_1.fq.gz"
```

Now run snakemake again:

In [ ]:
snakemake --cores 1

What did snakemake do?

- Snakemake will do nothing. Because By default, snakemake only runs the *first* rule in a Snakefile.

How can you bypass this?

- You can give a rule name on the command line, if you like, **or** you can tell snakemake what output file(s) you want.

Lets do the latter and run the following command:

In [ ]:
snakemake --cores 1 data/0Hour_001_1_fastqc.html data/6Hour_001_1_fastqc.html

Now you should see the second fastqc command run, with the appropriate output files!

Note that snakemake only runs the second rule, because it looks at the output files and sees that the first file you wanted, `0Hour_001_1_fastqc.html` already exists!

Its a good feature when you are dealing with only few rules or while debugging your rules. The downside here is:

* this is pretty long compared to the same shell script...
* specifying which file or rule you want is kind of annoying...

### A first refactoring: adding a better default rule

Let's start refactoring (cleaning up) this Snakefile.

First, let's add a rule at the top:

```
rule all:
  input:
    "data/0Hour_001_1_fastqc.html",
    "data/6Hour_001_1_fastqc.html"

rule fastqc_a_file:
  input:
    "data/0Hour_001_1.fq.gz"
  output:
    "data/0Hour_001_1_fastqc.html",
    "data/0Hour_001_1_fastqc.zip"
  shell:
    "fastqc data/0Hour_001_1.fq.gz"

rule fastqc_a_file2:
  input:
    "data/6Hour_001_1.fq.gz"
  output:
    "data/6Hour_001_1_fastqc.html",
    "data/6Hour_001_1_fastqc.zip"
  shell:
    "fastqc data/6Hour_001_1.fq.gz"
```

This rule, by convention called `all`, is a default rule that produces all the final output files. 

But it's a bit weird! It's all input, and no output!

Its a blank rule that gathers together all of the various files you want produced, and says "hey, snakemake, I depend on all of these files for my input - make them for me!" And then, once those files are all there, it ...does nothing.

Yep, this is perfectly legal in snakemake, and it's one way to make your life easier.

Note that `snakemake --cores 1 -f` no longer works properly, because `-f` only forces rerunning a single rule. 

To rerun everything, you can run `touch data/*.fq.gz` to make all the output files stale; or `rm data/*.html` to remove some of the output files.

### A second refactoring: doing a bit of templating

There's a lot of repetition in each of these rules. Let's collapse it down a little bit by replacing the filename in the fastqc command with a magic variable, `{input}`.

```
rule all:
  input:
    "data/0Hour_001_1_fastqc.html",
    "data/6Hour_001_1_fastqc.html"

rule fastqc_a_file:
  input:
    "data/0Hour_001_1.fq.gz"
  output:
    "data/0Hour_001_1_fastqc.html",
    "data/0Hour_001_1_fastqc.zip"
  shell:
    "fastqc {input}"

rule fastqc_a_file2:
  input:
    "data/6Hour_001_1.fq.gz"
  output:
    "data/6Hour_001_1_fastqc.html",
    "data/6Hour_001_1_fastqc.zip"
  shell:
    "fastqc {input}"
```

This all works as before, but now the rule is a bit more generic and will work with any input file. Sort of.

The input files data/0Hour_001_1.fq.gz and data/6Hour_001_1.fq.gz are still hardcoded, the variable {input} means take files mentioned in the input section.

